# PokéData: A Data-Driven Victory

__Context:__ Professor Oak's hypothesis is that pokémon have been getting stronger with each generation however some of his colleagues seem to disagree. With the pokémon championship battle coming closer, he wants to use this event to test his theory. He tasks a promising young trainer to fight the undefeated champion Cynthia with a carefully constructed team of 6 pokémon and bring an end to her win streak. 

__Objective:__ Create a team of 6 non-legendary pokémon that can defeat Cynthia's team and analyse trends in the data to in order to assess professor Oak's hypothesis.

## Setup: Cleaning the Data

In [22]:
# import libraries

import pandas as pd
import sqlite3

In [23]:
# loading dataset

df = pd.read_csv('C:/Users/danny/Documents/Projects/PokeData/MainStats.csv')
df.tail()

,ID,Name,Total,HP,Attack,Defense,SpAtk,SpDef,Speed,Type1,Type2,Height,Weight
1204,1021,Raging Bolt,590,125,73,91,137,89,75,Electric,Dragon,NaN,NaN
1205,1022,Iron Boulder,590,90,120,80,68,108,124,Rock,Psychic,NaN,NaN
1206,1023,Iron Crown,590,90,72,100,122,108,98,Steel,Psychic,NaN,NaN
1207,1024,Terapagos,450,90,65,85,65,85,60,Normal,NaN,NaN,NaN
1208,1025,Pecharunt,600,88,88,160,88,88,88,Poison,Ghost,NaN,NaN


This data is missing some key information that we will need such as the generation of each Pokémon and their legendary status. To fix this, we need to find the relevant data and append it to the above dataset.

In [24]:
# loading other dataset

df2 = pd.read_csv('C:/Users/danny/Documents/Projects/PokeData/SecondaryStats.csv')
df2.tail()

,number,name,type1,type2,total,hp,attack,defense,sp_attack,sp_defense,speed,generation,legendary
1067,896,Glastrier,Ice,NaN,580,100,145,130,65,110,30,8,True
1068,897,Spectrier,Ghost,NaN,580,100,65,60,145,80,130,8,True
1069,898,Calyrex,Psychic,Grass,500,100,80,80,80,80,80,8,True
1070,898,Ice Rider Calyrex,Psychic,Ice,680,100,165,150,85,130,50,8,True
1071,898,Shadow Rider Calyrex,Psychic,Ghost,680,100,85,80,165,100,150,8,True


In [25]:
# connecting to database and creating SQL tables

cnn = sqlite3.connect(':memory:')

df.to_sql('main_stats', cnn)
df2.to_sql('secondary_stats', cnn)

1072

In [26]:
# join tables 

query = '''
SELECT main_stats.ID, main_stats.Name, main_stats.HP, main_stats.Attack, main_stats.Defense, main_stats.SpAtk, main_stats.SpDef, main_stats.Speed,
main_stats.Type1, main_stats.Type2, secondary_stats.generation, secondary_stats.legendary
FROM main_stats
LEFT JOIN secondary_stats
ON main_stats.ID = secondary_stats.number;
'''

result = pd.read_sql_query(query, con=cnn)
result.tail()

,ID,Name,HP,Attack,Defense,SpAtk,SpDef,Speed,Type1,Type2,generation,legendary
1603,1021,Raging Bolt,125,73,91,137,89,75,Electric,Dragon,NaN,NaN
1604,1022,Iron Boulder,90,120,80,68,108,124,Rock,Psychic,NaN,NaN
1605,1023,Iron Crown,90,72,100,122,108,98,Steel,Psychic,NaN,NaN
1606,1024,Terapagos,90,65,85,65,85,60,Normal,None,NaN,NaN
1607,1025,Pecharunt,88,88,160,88,88,88,Poison,Ghost,NaN,NaN


The number of entries in both tables is different meaning that the generation and legendary columns in the result dataset contain missing values. Lets fix this.

In [27]:
# update tables to show generation and legendary status for all null values

result.to_sql('result', cnn)

query = '''
UPDATE result
SET legendary = True
WHERE ID IN (1001, 1002, 1003, 1004, 1007, 1008, 1014, 1015, 1016, 1017, 1024);
'''

query2 = '''
UPDATE result
SET legendary = False
WHERE legendary IS NULL
'''

# the first pokémon in generation 9 has an ID of 899
query3 = '''
UPDATE result
SET generation = 9
WHERE ID > 898
'''

cnn.execute(query)
cnn.execute(query2)
cnn.execute(query3)
cnn.commit()

test_query = '''
SELECT *
FROM result;
'''

filtered_result = pd.read_sql_query(test_query, con=cnn)
filtered_result.tail()

,index,ID,Name,HP,Attack,Defense,SpAtk,SpDef,Speed,Type1,Type2,generation,legendary
1603,1603,1021,Raging Bolt,125,73,91,137,89,75,Electric,Dragon,9.0,0.0
1604,1604,1022,Iron Boulder,90,120,80,68,108,124,Rock,Psychic,9.0,0.0
1605,1605,1023,Iron Crown,90,72,100,122,108,98,Steel,Psychic,9.0,0.0
1606,1606,1024,Terapagos,90,65,85,65,85,60,Normal,None,9.0,1.0
1607,1607,1025,Pecharunt,88,88,160,88,88,88,Poison,Ghost,9.0,0.0


In [29]:
# deleting duplicate values

delete = '''
DELETE FROM result
WHERE rowid NOT IN (
    SELECT MIN(rowid)
    FROM result
    GROUP BY ID
);
'''

cnn.execute(delete)
cnn.commit()

test_delete = '''
SELECT *
FROM result;
'''

pokemon = pd.read_sql_query(test_delete, con=cnn)
pokemon.to_sql('pokemon', cnn, if_exists='replace', index=False)
pokemon.tail()

,index,ID,Name,HP,Attack,Defense,SpAtk,SpDef,Speed,Type1,Type2,generation,legendary
1020,1603,1021,Raging Bolt,125,73,91,137,89,75,Electric,Dragon,9.0,0.0
1021,1604,1022,Iron Boulder,90,120,80,68,108,124,Rock,Psychic,9.0,0.0
1022,1605,1023,Iron Crown,90,72,100,122,108,98,Steel,Psychic,9.0,0.0
1023,1606,1024,Terapagos,90,65,85,65,85,60,Normal,None,9.0,1.0
1024,1607,1025,Pecharunt,88,88,160,88,88,88,Poison,Ghost,9.0,0.0


We can see now that the number of rows is 1025 which matches the number of pokémon that exist. There are no null values and all the columns have information that we need. The data is now cleaned and ready for us to work with.

## Intro: Querying the Data

In [30]:
# number of pokemon in each generation

query = '''
SELECT COUNT(ID) as pokemon_count, generation
FROM pokemon
GROUP BY generation;
'''

cnn.execute(query)
cnn.commit()

result = pd.read_sql_query(query, con=cnn)
print(result)

   pokemon_count  generation
0              2         0.0
1            151         1.0
2            100         2.0
3            135         3.0
4            107         4.0
5            156         5.0
6             72         6.0
7             86         7.0
8             89         8.0
9            127         9.0


We have found some errors in the data. 2 pokemon are classified as generation 0 which doesn't exist. Also, generations 8 and 9 do not have the correct count. Lets fix this.

In [31]:
# finding the error

query = '''
SELECT ID, Name
FROM pokemon
WHERE generation = 0;
'''

cnn.execute(query)
cnn.commit()

result = pd.read_sql_query(query, con=cnn)
result.head()

,ID,Name
0,808,Meltan
1,809,Melmetal


In [32]:
# fixing the problem

query = '''
UPDATE pokemon
SET generation = 7
WHERE ID IN (808, 809)
'''

cnn.execute(query)
cnn.commit()

test_query = '''
SELECT ID, Name, generation
FROM pokemon
WHERE ID IN (808, 809)
'''

filtered_result = pd.read_sql_query(test_query, con=cnn)
filtered_result.tail()

,ID,Name,generation
0,808,Meltan,7.0
1,809,Melmetal,7.0


In [33]:
# finding the error

query = '''
SELECT ID, Name, generation
FROM pokemon
WHERE generation = 9
LIMIT 10;
'''

cnn.execute(query)
cnn.commit()

result = pd.read_sql_query(query, con=cnn)
print(result)

    ID                      Name  generation
0  899                   Wyrdeer         9.0
1  900                   Kleavor         9.0
2  901                  Ursaluna         9.0
3  902          Basculegion Male         9.0
4  903                  Sneasler         9.0
5  904                  Overqwil         9.0
6  905  Enamorus Incarnate Forme         9.0
7  906                Sprigatito         9.0
8  907                 Floragato         9.0
9  908               Meowscarada         9.0


In [34]:
# fixing the error
# the pokemon with id ranging from 899 to 905 are classified as generation 9 rather than generation 8

query = '''
UPDATE pokemon
SET generation = 8
WHERE ID >= 899 AND ID <= 905
'''

cnn.execute(query)
cnn.commit()

test_query = '''
SELECT ID, Name, generation
FROM pokemon
WHERE ID >= 899 AND ID <= 905
'''

filtered_result = pd.read_sql_query(test_query, con=cnn)
print(filtered_result)

    ID                      Name  generation
0  899                   Wyrdeer         8.0
1  900                   Kleavor         8.0
2  901                  Ursaluna         8.0
3  902          Basculegion Male         8.0
4  903                  Sneasler         8.0
5  904                  Overqwil         8.0
6  905  Enamorus Incarnate Forme         8.0


In [35]:
# rerunning the query to find count of all pokemon per generation

query = '''
SELECT COUNT(ID) AS pokemon_count, generation
FROM pokemon
GROUP BY generation;
'''

result = pd.read_sql_query(query, con=cnn)
print(result)

# save cleaned data to use with power BI
pokemon_df = pd.read_sql_query('SELECT * FROM pokemon', con=cnn)
pokemon_df.to_csv('cleaned_data.csv', index=False)

   pokemon_count  generation
0            151         1.0
1            100         2.0
2            135         3.0
3            107         4.0
4            156         5.0
5             72         6.0
6             88         7.0
7             96         8.0
8            120         9.0


In [36]:
# Create a new table without the unwanted column
cnn.execute('''
CREATE TABLE temp_pokemon AS
SELECT ID, Name, HP, Attack, Defense, SpAtk, SpDef, Speed, Type1, Type2, generation, legendary
FROM pokemon;
''')

# Drop the old pokemon table
cnn.execute('DROP TABLE pokemon;')

# Rename the new table
cnn.execute('ALTER TABLE temp_pokemon RENAME TO pokemon;')

# Commit the changes
cnn.commit()

pokemon.head()

,index,ID,Name,HP,Attack,Defense,SpAtk,SpDef,Speed,Type1,Type2,generation,legendary
0,0,1,Bulbasaur,45,49,49,65,65,45,Grass,Poison,1.0,0.0
1,1,2,Ivysaur,60,62,63,80,80,60,Grass,Poison,1.0,0.0
2,2,3,Venusaur,80,82,83,100,100,80,Grass,Poison,1.0,0.0
3,8,4,Charmander,39,52,43,60,50,65,Fire,None,1.0,0.0
4,9,5,Charmeleon,58,64,58,80,65,80,Fire,None,1.0,0.0


In [37]:
# finding the count of pokemon per type

query = '''
WITH CombinedTypes AS (
    SELECT Type1 AS type
    FROM pokemon
    UNION ALL
    SELECT Type2 AS type
    FROM pokemon
    WHERE Type2 IS NOT NULL
)

SELECT type, COUNT(*) AS pokemon_count
FROM CombinedTypes
GROUP BY type;
'''

cnn.execute(query)
cnn.commit()

result = pd.read_sql_query(query, con=cnn)
print(result)

        type  pokemon_count
0        Bug             92
1       Dark             69
2     Dragon             70
3   Electric             69
4      Fairy             64
5   Fighting             73
6       Fire             82
7     Flying            109
8      Ghost             65
9      Grass            127
10    Ground             75
11       Ice             48
12    Normal            131
13    Poison             83
14   Psychic            102
15      Rock             74
16     Steel             65
17     Water            154


In [40]:
# finding the average total stats for each generation

query = '''
SELECT 
    generation, 
    ROUND(AVG(total_stats), 2) AS avg_total_stats
FROM (
    SELECT 
        generation, 
        (HP + Attack + Defense + SpAtk + SpDef + Speed) AS total_stats
    FROM 
        pokemon
) AS stats_per_pokemon
GROUP BY 
    generation;
'''

cnn.execute(query)
cnn.commit()

result = pd.read_sql_query(query, con=cnn)
print(result)

   generation  avg_total_stats
0         1.0           407.64
1         2.0           407.18
2         3.0           403.73
3         4.0           445.57
4         5.0           425.76
5         6.0           429.31
6         7.0           449.41
7         8.0           439.22
8         9.0           457.39


In [44]:
# average of each stat per generation

query = '''
SELECT generation, ROUND(AVG(HP) , 2) AS avg_hp, ROUND(AVG(Attack) , 2) AS avg_attack, ROUND(AVG(Defense) , 2) AS avg_defense,
ROUND(AVG(SpAtk) , 2) AS avg_spatk, ROUND(AVG(SpDef) , 2) AS avg_spdef, ROUND(AVG(Speed), 2) AS avg_speed 
FROM pokemon
GROUP BY generation;
'''

cnn.execute(query)
cnn.commit()

result = pd.read_sql_query(query, con=cnn)
print(result)

   generation  avg_hp  avg_attack  avg_defense  avg_spatk  avg_spdef  \
0         1.0   64.21       72.91        68.23      67.14      66.09   
1         2.0   70.98       68.26        69.69      64.50      72.34   
2         3.0   65.67       73.11        69.01      67.86      66.47   
3         4.0   73.10       80.21        75.11      73.28      74.38   
4         5.0   70.31       81.03        71.24      69.24      67.33   
5         6.0   68.92       72.50        75.08      72.54      74.58   
6         7.0   71.01       84.77        78.73      74.95      74.57   
7         8.0   72.82       82.91        73.28      71.65      69.61   
8         9.0   77.39       82.43        76.78      72.87      72.48   

   avg_speed  
0      69.07  
1      61.41  
2      61.61  
3      69.48  
4      66.60  
5      65.68  
6      65.38  
7      68.95  
8      75.45  
